In [ ]:
import os
from PublicDataReader import TransactionPrice
service_key = os.getenv("PUBLIC_DATA_HUB")
api = TransactionPrice(service_key)

df = api.get_data(
    property_type="아파트",
    trade_type="매매",
    sigungu_code="11650",
    start_year_month="202401",
    end_year_month="202412",
)

# df.to_csv("아파트_매매_202501_202506.csv", index=False)
df

In [29]:
import PublicDataReader as pdr
sigungu_name = "강남구"
code = pdr.code_bdong()
code.loc[(code['시군구명'].str.contains(sigungu_name)) &
         (code['읍면동명']=='')]

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.8.1. 시행)


,시도코드,시도명,시군구코드,시군구명,법정동코드,읍면동명,동리명,생성일자,말소일자
1003,11,서울특별시,11680,강남구,1168000000,,,19880423,


In [2]:
import os
from PublicDataReader import Kosis


# KOSIS 공유서비스 Open API 사용자 인증키
service_key = os.getenv("KOSIS")

# 인스턴스 생성하기
api = Kosis(service_key)

In [10]:
df1 = api.get_data(
    "KOSIS통합검색",
    searchNm="아파트거래통계",
    )
df1.head(1)

,기관ID,기관명,통계표ID,통계표명,조사ID,조사명,KOSIS목록구분,KOSIS통계표위치,통계표위치,통계표주요내용,수록기간시작일,수록기간종료일,통계표주석,추천통계표여부,KOSIS목록URL,KOSIS통계표URL,검색결과건수,검색어명
0,408,한국부동산원,DT_KAB_11672_S25,아파트 전세 실거래 평균가격,2009045,공동주택실거래가격지수,MT_ZTITLE,주거 > 공동주택실거래가격지수,I1 > I1_1,행정구역별 가격 광역시 부산 제주 광주 충남 전북 서울 서북권 도 충북 전남 경남 ...,2014,2025,*공동주택 실거래가격지수 중 전세 실거래가격지수는 확정일자를 기초로 신규계약 및 재...,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=408...,372,아파트거래통계


In [13]:
df2 = api.get_data(
    "통계표설명",
    "분류항목",
    orgId = "408",
    tblId = "DT_KAB_11672_S25",
    metaItm = "ALL",
    )
df2.to_csv("아파트거래통계.csv")